### Скрипты построения таблиц look-alike модели для Clickhouse



In [34]:
sc.stop() # Освобождаем ненужные ресурсы

In [35]:
## Генерация фрагментов url

def id_up_query(source_table,id_up_table, batch_num = 1):
    id_up_query = ''
    

    s1 = '''
    INSERT INTO #id_up SELECT 
        ymd,
        uid,
        replaceRegexpOne(url, '^([^/?#]+)/.*', '\\\\1') AS up,
        toUInt32(count()) as cnt,
        toUInt32(max(ts) - min(ts)) as duration
    FROM #source_table
    WHERE match(url, '^([^/?#]+).*') AND uid > '#low' and uid <= '#high'
    GROUP BY ymd, uid, replaceRegexpOne(url, '^([^/?#]+)/.*', '\\\\1') ;\n'''
    
    s2 = '''
    INSERT INTO #id_up SELECT
        ymd,
        uid,
        replaceRegexpOne(url, '^([^/?#]+)/([^/?#]+).*', '\\\\1[0]\\\\2') AS up,
        toUInt32(count()) as cnt,
        toUInt32(max(ts) - min(ts)) as duration
    FROM #source_table
    WHERE match(url, '^([^/?#]+)/([^/?#]+).*')  AND uid > '#low' and uid <= '#high'
    GROUP BY ymd, uid, replaceRegexpOne(url, '^([^/?#]+)/([^/?#]+).*', '\\\\1[0]\\\\2') ;\n'''

    
    s3 = '''
    INSERT INTO #id_up SELECT
        ymd,
        uid,
        replaceRegexpOne(url, '^([^/?#]+)/([^/?#]+)/([^/?#]+).*', '\\\\1[1]\\\\3')  AS up,
        toUInt32(count()) as cnt,
        toUInt32(max(ts) - min(ts)) as duration
    FROM #source_table
    WHERE match(url, '^([^/?#]+)/([^/?#]+)/([^/?#]+).*') AND uid > '#low' and uid <= '#high'
    GROUP BY ymd, uid, replaceRegexpOne(url, '^([^/?#]+)/([^/?#]+)/([^/?#]+).*', '\\\\1[1]\\\\3') ;\n'''

    
    s4 = '''
    INSERT INTO #id_up SELECT
        ymd,
        uid,
        replaceRegexpOne(url, '^([^/?#]+)/([^/?#]+)/([^/?#]+)/([^/?#]+).*', '\\\\1[2]\\\\4')  AS up,
        toUInt32(count()) as cnt,
        toUInt32(max(ts) - min(ts)) as duration
    FROM #source_table
    WHERE match(url, '^([^/?#]+)/([^/?#]+)/([^/?#]+)/([^/?#]+).*') AND uid > '#low' and uid <= '#high'
    GROUP BY ymd, uid, replaceRegexpOne(url, '^([^/?#]+)/([^/?#]+)/([^/?#]+)/([^/?#]+).*', '\\\\1[2]\\\\4') ;\n'''

    
    id_up_query += 'DROP TABLE IF EXISTS #id_up;\n'
    id_up_query +='CREATE TABLE #id_up (ymd Date, uid String, up String, cnt UInt32, duration UInt32) ' + \
           'ENGINE = MergeTree(ymd, (uid, up, ymd), 8192);\n'
    
    N = batch_num - 1
    for i in range(N+1):
        low, high = high if i>0 else '', hex(16*16*(i+N)/N)[-2:].upper() if i<N else 'ZZ'
        id_up_query += s1.replace('#low',low).replace('#high',high)
        id_up_query += s2.replace('#low',low).replace('#high',high)
        id_up_query += s3.replace('#low',low).replace('#high',high)
        id_up_query += s4.replace('#low',low).replace('#high',high)
    
    id_up_query = id_up_query.replace('#id_up',id_up_table).replace('#source_table',source_table)
    return id_up_query

#print(id_up_query(train_date,source_table,id_up_table, batch_num = 128))

In [36]:

def pos_id_query(pos_id_table, target_url,source_table):
    # Положительные куки. Маленькая табличка.
    posit_query = '''
    DROP TABLE IF EXISTS #pos_id;
    
    CREATE TABLE #pos_id ENGINE =  TinyLog as
    SELECT DISTINCT
       uid
    FROM
       #source_table
    WHERE
       url like '%#target_url%'
    ;
    '''.replace('#pos_id', pos_id_table).replace('#source_table',source_table).replace('#target_url',target_url)
    return(posit_query)

#print(pos_id_query(pos_id_table, target_url))

In [37]:

def up_query(up_table, pos_id_table, exclude_url, id_up_table,batch_num = 1):
    up_query = '''
    DROP TABLE IF EXISTS #up_table;
    CREATE TABLE #up_table 
       (ymd Date, up String, total UInt32, positive UInt16, score Int32)
    ENGINE = MergeTree(ymd, (up, ymd), 8192);
    '''
    
    q = '''
    INSERT INTO #up_table
    SELECT
        ymd,
        up,
        toUInt32(total) as total,
        toUInt16(positive) as positive,
        toInt32(100000 * log((positive + 0.1)/(total - positive + 0.1))) as score
    FROM
       (SELECT 
           ymd,
           up, 
           count() as total,
           sum(if(uid in (select uid from #pos_id_table), 1, 0)) as positive
       FROM
          #id_up_table
       WHERE 
          NOT up like '%#exclude_url%'
          AND cityHash64(up) % #N = #rem
       GROUP BY 
          ymd, up
       )
    WHERE
       total > 30000
       OR (positive > 1 and total > 20)
    ;
    '''


    for i in range(batch_num):
        up_query += q.replace('#N',str(batch_num)).replace('#rem',str(i))
    up_query = up_query.replace('#pos_id_table',pos_id_table) \
       .replace('#exclude_url',exclude_url).replace('#id_up_table',id_up_table).replace('#up_table',up_table)
    return up_query

#print(up_query(up_table, pos_id_table, target_url, exclude_url, id_up_table,batch_num = 32))

In [38]:
def up_full_query(up_table, pos_id_table, exclude_url, id_up_table,batch_num = 1):
    up_query = '''
    DROP TABLE IF EXISTS #up_table;
    CREATE TABLE #up_table 
       (ymd Date, up String, total UInt32, positive UInt16, score Int32)
    ENGINE = MergeTree(ymd, (up, ymd), 8192);
    '''
    
    q = '''
    INSERT INTO #up_table
    SELECT
        ymd,
        up,
        toUInt32(total) as total,
        toUInt16(positive) as positive,
        toInt32(100000 * log((positive + 0.1)/(total - positive + 0.1))) as score
    FROM
       (SELECT 
           ymd,
           up, 
           count() as total,
           sum(if(uid in (select uid from #pos_id_table), 1, 0)) as positive
       FROM
          #id_up_table
       WHERE 
          NOT up like '#exclude_url%'
          AND cityHash64(up) % #N = #rem
       GROUP BY 
          ymd, up
       )
    WHERE
       total > 20
       OR positive > 0
    ;
    '''


    for i in range(batch_num):
        up_query += q.replace('#N',str(batch_num)).replace('#rem',str(i))
    up_query = up_query.replace('#pos_id_table',pos_id_table) \
       .replace('#exclude_url',exclude_url).replace('#id_up_table',id_up_table).replace('#up_table',up_table)
    return up_query

#print(up_query(up_table, pos_id_table, target_url, exclude_url, id_up_table,batch_num = 32))

In [39]:
def id_old_query(id_table, id_up_table,up_table,pos_id_table, batch_num):
    mv = str(-1150000) # Neutral value for vk.com or yandex.ru 
    idq = '''    
    DROP TABLE IF EXISTS #id_table;
    CREATE TABLE #id_table 
       (ymd Date, uid String, label UInt8, smax Int32, savg Int32, ssum Int32, smedian Int32, sstd UInt16, 
        cntrepeat UInt32, cntuniq UInt32, duration UInt64, has_scores UInt8, mobile UInt8, emailru UInt8, vkru UInt8, okru UInt8, social_other UInt8, 
        s1 Int32, s2 Int32,s3 Int32, s4 Int32, s5 Int32, s6 Int32, s7 Int32, s8 Int32, s9 Int32, s10 Int32, 
        sm1 Int32, sm2 Int32, sm3 Int32, sm4 Int32, sm5 Int32
       )
    ENGINE = MergeTree(ymd, (uid, ymd), 8192);
    '''

    q = '''
    INSERT INTO #id_table
    SELECT
        ymd,
        uid,
        (uid IN (SELECT uid from #pos_id_table)) as label,
        toInt32(max(score)) as smax,
        toInt32(avg(score)) as savg,
        toInt32(sum(score)) as ssum,
        toInt32(median(score)) as smedian,
        toUInt16(stddevSamp(score)) as sstd,
        toUInt32(sum(cnt)) as cntrepeat,
        toUInt32(count()) as cntuniq,
        toUInt64(sum(duration)) as duration,
        toUInt8(sum(has_scores)) as has_scores,
        toUInt8(max(mobile)) as mobile,
        toUInt8(max(emailru)) as emailru,
        toUInt8(max(vkru)) as vkru,
        toUInt8(max(okru)) as okru,
        toUInt8(max(social_other)) as social_other,
        length(groupArray(score) as sl) >= 1 ? sl[1] : toInt32(#mv) as s1,
        length(sl) >= 2 ? sl[2] : toInt32(#mv) as s2,
        length(sl) >= 3 ? sl[3] : toInt32(#mv) as s3,
        length(sl) >= 4 ? sl[4] : toInt32(#mv) as s4,
        length(sl) >= 5 ? sl[5] : toInt32(#mv) as s5,
        length(sl) >= 6 ? sl[6] : toInt32(#mv) as s6,
        length(sl) >= 7 ? sl[7] : toInt32(#mv) as s7,
        length(sl) >= 8 ? sl[8] : toInt32(#mv) as s8,
        length(sl) >= 9 ? sl[9] : toInt32(#mv) as s9,
        length(sl) >= 10 ? sl[10] : toInt32(#mv) as s10,
        length(sl) >= 1 ? sl[-1] : toInt32(#mv) as sm1,
        length(sl) >= 2 ? sl[-2] : toInt32(#mv) as sm2,
        length(sl) >= 3 ? sl[-3] : toInt32(#mv) as sm3,
        length(sl) >= 4 ? sl[-4] : toInt32(#mv) as sm4,
        length(sl) >= 5 ? sl[-5] : toInt32(#mv) as sm5        
    FROM        
        (SELECT
            ymd,
            uid,
            (score=0) and (total = 0) ? toInt32(#mv) : score as score,
            cnt,
            (total > 0) ? 1 : 0 as has_scores,
            duration,
            (up like 'm.%') as mobile,
            (up like '%e.mail.ru%') as emailru,
            match(up,'^vk\\.com|[^A-Za-z]vk\\.com|^vk.me|[^A-Za-z]vk\\.me|^vk\\.cc|[^A-Za-z]vk\\.cc|vkontakte\\.') as vkru,
            match(up,'^ok\\.ru|[^A-Za-z]ok\\.ru|odnoklassniki\\.ru') as okru,
            match(up,'^fb\\.com|[^A-Za-z]fb\\.com|instagram\\.com|twitter\\.com|my\\.mail\\.ru|livejournal\\.com|^lj\\.ru') as social_other
        FROM
            (select * from #id_up_table where uid > '#low' and uid <= '#high')
        ANY LEFT JOIN (select ymd,up,score,total from #up_table) USING (ymd,up) 
        ORDER BY uid,score DESC
        )
    GROUP BY ymd,uid
    ;        
    '''
    
    N = batch_num - 1
    for i in range(N+1):
        low, high = high if i>0 else '', hex(16*16*(i+N)/N)[-2:].upper() if i<N else 'G'
        idq += q.replace('#low',low).replace('#high',high)
    
    idq = idq.replace('#id_table',id_table).replace('#id_up_table',id_up_table).replace('#up_table',up_table) \
        .replace('#pos_id_table',pos_id_table).replace('#mv', mv)
    return idq



In [40]:
def id_query(id_table, id_up_table,up_table,pos_id_table, batch_num):
    mv = str(-1150000) # Neutral value for vk.com or yandex.ru 
    idq = '''    
    DROP TABLE IF EXISTS #id_table;
    CREATE TABLE #id_table 
       (ymd Date, uid String, label UInt8, smax Int32, savg Int32, ssum Int32, smedian Int32, sstd UInt16, 
        cntrepeat UInt32, cntuniq UInt32, duration UInt64, has_scores UInt8, mobile UInt8, emailru UInt8, vkru UInt8, okru UInt8, social_other UInt8, 
        s1 Int32, s2 Int32,s3 Int32, s4 Int32, s5 Int32, s6 Int32, s7 Int32, s8 Int32, s9 Int32, s10 Int32, 
        sm1 Int32, sm2 Int32, sm3 Int32, sm4 Int32, sm5 Int32
       )
    ENGINE = MergeTree(ymd, (uid, ymd), 8192);
    '''

    q = '''
    INSERT INTO #id_table
    SELECT
        ymd,
        uid,
        (uid IN (SELECT uid from #pos_id_table)) as label,
        toInt32(max(score)) as smax,
        toInt32(ssum/has_scores) as savg,
        toInt32(sum(score * has_score)) as ssum,
        toInt32(median(score)) as smedian,
        toUInt16(stddevSamp(score)) as sstd,
        toUInt32(sum(cnt)) as cntrepeat,
        toUInt32(count()) as cntuniq,
        toUInt64(sum(duration)) as duration,
        toUInt8(sum(has_score)) as has_scores,
        toUInt8(max(mobile)) as mobile,
        toUInt8(max(emailru)) as emailru,
        toUInt8(max(vkru)) as vkru,
        toUInt8(max(okru)) as okru,
        toUInt8(max(social_other)) as social_other,
        length(groupArray(score) as sl) >= 1 ? sl[1] : toInt32(#mv) as s1,
        length(sl) >= 2 ? sl[2] : toInt32(#mv) as s2,
        length(sl) >= 3 ? sl[3] : toInt32(#mv) as s3,
        length(sl) >= 4 ? sl[4] : toInt32(#mv) as s4,
        length(sl) >= 5 ? sl[5] : toInt32(#mv) as s5,
        length(sl) >= 6 ? sl[6] : toInt32(#mv) as s6,
        length(sl) >= 7 ? sl[7] : toInt32(#mv) as s7,
        length(sl) >= 8 ? sl[8] : toInt32(#mv) as s8,
        length(sl) >= 9 ? sl[9] : toInt32(#mv) as s9,
        length(sl) >= 10 ? sl[10] : toInt32(#mv) as s10,
        length(sl) >= 1 ? sl[-1] : toInt32(#mv) as sm1,
        length(sl) >= 2 ? sl[-2] : toInt32(#mv) as sm2,
        length(sl) >= 3 ? sl[-3] : toInt32(#mv) as sm3,
        length(sl) >= 4 ? sl[-4] : toInt32(#mv) as sm4,
        length(sl) >= 5 ? sl[-5] : toInt32(#mv) as sm5        
    FROM        
        (SELECT
            ymd,
            uid,
            (score=0) and (total = 0) ? toInt32(#mv) : score as score,
            cnt,
            (total > 0) ? 1 : 0 as has_score,
            duration,
            (up like 'm.%') as mobile,
            (up like '%e.mail.ru%') as emailru,
            match(up,'^vk\\.com|[^A-Za-z]vk\\.com|^vk.me|[^A-Za-z]vk\\.me|^vk\\.cc|[^A-Za-z]vk\\.cc|vkontakte\\.') as vkru,
            match(up,'^ok\\.ru|[^A-Za-z]ok\\.ru|odnoklassniki\\.ru') as okru,
            match(up,'^fb\\.com|[^A-Za-z]fb\\.com|instagram\\.com|twitter\\.com|my\\.mail\\.ru|livejournal\\.com|^lj\\.ru') as social_other
        FROM
            (select * from #id_up_table where uid > '#low' and uid <= '#high')
        ANY LEFT JOIN (select up,score,total from #up_table) USING (up) 
        ORDER BY uid,score DESC
        )
    GROUP BY ymd,uid
    ;        
    '''
    
    N = batch_num - 1
    for i in range(N+1):
        low, high = high if i>0 else '', hex(16*16*(i+N)/N)[-2:].upper() if i<N else 'G'
        idq += q.replace('#low',low).replace('#high',high)
    
    idq = idq.replace('#id_table',id_table).replace('#id_up_table',id_up_table).replace('#up_table',up_table) \
        .replace('#pos_id_table',pos_id_table).replace('#mv', mv)
    return idq



In [41]:
def id_threshold_tuning_query(id_table, id_up_table,up_table,pos_id_table, batch_num):
    mv = str(-1250000) # Neutral value for vk.com or yandex.ru 
    idq = '''    
    DROP TABLE IF EXISTS #id_table;
    CREATE TABLE #id_table 
       (ymd Date, uid String, label UInt8, has_scores UInt8, scorelist Array(Int32), 
       totallist Array(UInt32), poslist Array(UInt16)
       )
    ENGINE = MergeTree(ymd, (uid, ymd), 8192);
    '''

    q = '''
    INSERT INTO #id_table
    SELECT
        ymd,
        uid,
        (uid IN (SELECT uid from #pos_id_table)) as label,
        toUInt8(sum(has_score)) as has_scores,
        groupArray(score) as scorelist,
        groupArray(total) as totallist,
        groupArray(positive) as poslist
    FROM        
        (SELECT
            ymd,
            uid,
            (score=0) and (total = 0) ? toInt32(#mv) : score as score,
            cnt,
            (total > 0) ? 1 : 0 as has_score,
            total,
            positive
        FROM
            (select * from #id_up_table where uid > '#low' and uid <= '#high')
        ANY LEFT JOIN (select up,score,total,positive from #up_table) USING (up) 
        ORDER BY uid,score DESC
        )
    GROUP BY ymd,uid
    ;        
    '''
    
    N = batch_num - 1
    for i in range(N+1):
        low, high = high if i>0 else '', hex(16*16*(i+N)/N)[-2:].upper() if i<N else 'G'
        idq += q.replace('#low',low).replace('#high',high)
    
    idq = idq.replace('#id_table',id_table).replace('#id_up_table',id_up_table).replace('#up_table',up_table) \
        .replace('#pos_id_table',pos_id_table).replace('#mv', mv)
    return idq


In [42]:
def test_query(date, source_table, id_up_table , pos_id_table ,up_table, id_table):
    q = '''
    select '#source_table' as name, '#date' as date, toUInt64(count()) as cnt,toUInt64(uniq(uid)) as uid_cnt, toUInt64(uniq(url)) as up_cnt 
    from #source_table
    union all
    select '#id_up_table' as name, '#date' as date, toUInt64(count()) as cnt,toUInt64(uniq(uid)) as uid_cnt, toUInt64(uniq(up)) as up_cnt 
    from #id_up_table
    union all
    select '#up_table' as name, '#date' as date, toUInt64(count()) as cnt,toUInt64(0) as uid_cnt, toUInt64(uniq(up)) as up_cnt 
    from #up_table
    union all
    select '#id_table' as name, '#date' as date, toUInt64(count()) as cnt, toUInt64(uniq(uid)) as uid_cnt, toUInt64(0) as up_cnt 
    from #id_table
    union all
    select '#pos_id_table' as name, '#date' as date, toUInt64(count()) as cnt, toUInt64(uniq(uid)) as uid_cnt, toUInt64(0) as up_cnt 
    from #pos_id_table    
    format TabSeparatedWithNames
    ;
    '''.replace('#id_table',id_table).replace('#id_up_table',id_up_table).replace('#up_table',up_table) \
        .replace('#pos_id_table',pos_id_table).replace('#source_table',source_table).replace('#date',date)
    return q

In [43]:
#date = '20160817'
#source_table = 'rlog_' + date
#id_up_table = 'kosm.id_up_' + date
#target_url = 'raiffeisen.ru/retail/cards/credit'
#pos_id_table = 'kosm.id_posit_' + date
#exclude_url = 'raiffeisen.ru'
#up_table = 'kosm.up_' + date
#id_table = 'kosm.id_'+ date

def la_model_queries(date, target_url, exclude_url = '', source_table = '', id_up_table = '', \
                    pos_id_table = '',up_table = '',id_table='',train = False):
    if not source_table: source_table = 'rlog_' + date
    if not id_up_table: id_up_table = 'kosm.id_up_' + date
    if not pos_id_table: pos_id_table = 'kosm.id_posit_' + date
    if not exclude_url: exclude_url = target_url.split('/')[0]
    if not up_table: up_table = 'kosm.up_' + date
    if not id_table: id_table = 'kosm.id_'+ date
    res = ''
    res += id_up_query(source_table,id_up_table, batch_num = 128)
    res += pos_id_query(pos_id_table, target_url,source_table)
    if train: res += up_query(up_table, pos_id_table, exclude_url, id_up_table,batch_num = 32)
    res += id_query(id_table, id_up_table,up_table,pos_id_table, batch_num = 64)
    res += test_query(date, source_table, id_up_table , pos_id_table ,up_table, id_table)
    return res

#open('la_ch_queries_20160817.txt','w').write(la_model_queries('20160817', 'raiffeisen.ru/retail/cards/credit'))

In [44]:
def test_on_applicants(date,pos_id_table, exclude_url,id_up_table = '', up_table = '',id_table=''):
    if not id_up_table: id_up_table = 'kosm.id_up_' + date
    
    if not id_table: id_table = 'kosm.id_app_'+ date
    res = ''
    #res += up_query(up_table, pos_id_table, exclude_url, id_up_table,batch_num = 32)
    res += id_query(id_table, id_up_table,up_table,pos_id_table, batch_num = 64)
    #res += test_query(date, source_table, id_up_table , pos_id_table ,up_table, id_table)
    return res


In [45]:
dates = ['20160817','20160818','20160824']
exclude_url = 'tinkoff.ru'
queries_text = ''
for d in dates:
    pos_id_table = "(select uid from kosm.tnkf_app_id_20160817_24 where ymd = '" + d[:4] + '-' + d[4:6] + '-' + d[6:] + "')"
    if d == '20160817':
        queries_text += up_query('kosm.up_app_20160817', pos_id_table, exclude_url, 'kosm.id_up_20160817',batch_num = 32)
    queries_text += test_on_applicants(d, pos_id_table, exclude_url, up_table = 'kosm.up_app_20160817')

#open('lam_app_queries_{0}.txt'.format('_'.join(dates)),'w').write(queries_text)

In [46]:
#Даты для расчета. Исходные данные должны находится в таблицах cl-h вида rlog_дата.
dates = ['20160817','20160818','20160824'] #['20160828']
#target_url = 'raiffeisen.ru/retail/cards/credit'
target_url = 'mkb.ru/facility/private_person/cards/credit_card'
queries_text = ''
for d in dates:
    if d == '20160817':
        queries_text += up_full_query(up_table = 'kosm.up_full_20160817', pos_id_table = pos_id_table, 
                                      exclude_url = 'raiffeisen.ru', id_up_table = 'kosm.id_up_20160817',batch_num = 32)
    queries_text += la_model_queries(
        d, 
        target_url, 
        #source_table = '(select * from rlog where ymd = "{0}")'.format('2016-08-28'),
        up_table = 'kosm.up1_20160817', 
        train = (d == '20160817')
    )
    #queries_text += id_threshold_tuning_query('kosm.id_thres_' + d,'kosm.id_up_'+d,'kosm.up_full_20160817','kosm.id_posit_'+d,32*3)
    
# Генерация запросов в cl-house и запись их в файл.
open('lam_queries_{0}.txt'.format('_'.join(dates)),'w').write(queries_text)

#Далее необходимо переписать файл на сервер clickhouseв папку со скриптом lamtgen.sh и запустить его с 2 аргументами - имя 
# полученного файла и имя файла для записи логов.

In [47]:
a  =  '''# Проверка корректности разбиения
import itertools
import pandas as pd
h = list('0123456789ABCDEF')

N = 20
df = pd.DataFrame(index = [''.join(h2) for h2 in itertools.product(h,h)],columns = range(N+1))
df['k'] = df.index
for j in range(N+1):
        low, high = high if j>0 else '', hex(16*16*(j+N)/N)[-2:].upper() if j<N else 'ZZ'
        df[j] = df['k'].apply(lambda k: (k+'23'>low)*(k+'23'<=high))
            
print('Error rows: ' + str(df.loc[df.sum(axis=1)!=1]))
'''